# Binary Classification Deep Learning Model for [Project Name] Using Keras Version 2
### David Lowe
### October 11, 2019
Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: [Sample Paragraph - The purpose of this project is to construct a predictive model using various machine learning algorithms and to document the end-to-end steps using a template. The Pima Indians Diabetes dataset is a binary classification situation where we are trying to predict one of the two possible outcomes.]

INTRODUCTION: [Sample Paragraph - This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.]

ANALYSIS: [Sample Paragraph - The baseline performance of the model achieved an average accuracy score of 69.11%. Using the same training parameters, the model processed the test dataset with an accuracy of 78.12%, which was even better than results from the training data.]

CONCLUSION: [Sample Paragraph - For this dataset, the model built using Keras and TensorFlow achieved a satisfactory result and should be considered for future modeling activities]

Dataset Used: [Pima Indians Diabetes Data Set]

Dataset ML Model: Binary classification with numerical attributes

Dataset Reference: [https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv]

One potential source of performance benchmarks: [https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/]

Any deep-learning modeling project genrally can be broken down into about seven major tasks:
0. Prepare Environment
1. Load Data
2. Define Model
3. Compile Model
4. Fit Model
5. Evaluate Model
6. Finalize Model

# Section 0. Prepare Environment

In [1]:
# Create the random seed numbers for reproducible results
seedNum = 88

In [2]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import os
import smtplib
from datetime import datetime
from email.message import EmailMessage
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [3]:
# Configure a new global `tensorflow` session
import keras as K
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
tf.set_random_seed(seedNum)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.backend.set_session(sess)

Using TensorFlow backend.


In [4]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set the flag for splitting the dataset
splitDataset = True
splitPercentage = 0.25

# Set various default Keras modeling parameters
kernel_init = K.initializers.glorot_uniform(seed=seedNum)
loss = 'binary_crossentropy'
optimizer = 'adam'
metrics = ['accuracy']
epochs = 150
batches = 10

# Set the number of folds for cross validation
folds = 10

In [5]:
# Set up the email notification function
def email_notify(msg_text):
    sender = os.environ.get('MAIL_SENDER')
    receiver = os.environ.get('MAIL_RECEIVER')
    gateway = os.environ.get('SMTP_GATEWAY')
    smtpuser = os.environ.get('SMTP_USERNAME')
    password = os.environ.get('SMTP_PASSWORD')
    if sender==None or receiver==None or gateway==None or smtpuser==None or password==None:
        sys.exit("Incomplete email setup info. Script Processing Aborted!!!")
    msg = EmailMessage()
    msg.set_content(msg_text)
    msg['Subject'] = 'Notification from Keras Binary Classification Script'
    msg['From'] = sender
    msg['To'] = receiver
    server = smtplib.SMTP(gateway, 587)
    server.starttls()
    server.login(smtpuser, password)
    server.send_message(msg)
    server.quit()

In [6]:
if (notifyStatus): email_notify("Phase 0 Prepare Environment completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 1. Load Data

In [7]:
if (notifyStatus): email_notify("Phase 1 Load Data has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [8]:
# Load the dataset
dataset = np.loadtxt('pima-indians-diabetes.data.csv', delimiter=',')
print(dataset)

[[  6.    148.     72.    ...   0.627  50.      1.   ]
 [  1.     85.     66.    ...   0.351  31.      0.   ]
 [  8.    183.     64.    ...   0.672  32.      1.   ]
 ...
 [  5.    121.     72.    ...   0.245  30.      0.   ]
 [  1.    126.     60.    ...   0.349  47.      1.   ]
 [  1.     93.     70.    ...   0.315  23.      0.   ]]


In [9]:
# Split the original dataset into input (X) and output (y) variables
X_original = dataset[:,0:8]
y_original = dataset[:,8]
print('Shape of X_original:', X_original.shape, '| Shape of y_original:', y_original.shape)

Shape of X_original: (768, 8) | Shape of y_original: (768,)


In [10]:
# Split the data further into training and test datasets
if (splitDataset):
    X_train, X_test, y_train, y_test = train_test_split(X_original, y_original, test_size=splitPercentage, random_state=seedNum)
else:
    X_train, y_train = X_original, y_original
    X_test, y_test = X_original, y_original
print('Shape of X_train:', X_train.shape, '| Shape of y_train:', y_train.shape)
print('Shape of X_test:', X_test.shape, '| Shape of y_test:', y_test.shape)

Shape of X_train: (576, 8) | Shape of y_train: (576,)
Shape of X_test: (192, 8) | Shape of y_test: (192,)


In [11]:
if (notifyStatus): email_notify("Phase 1 Load Data completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 2. Define Model

In [12]:
if (notifyStatus): email_notify("Phase 2 Define Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [13]:
# Define the Keras model required for KerasClassifier
def create_model():
    model = K.models.Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer=kernel_init, activation='relu'))
    model.add(Dense(8, kernel_initializer=kernel_init, activation='relu'))
    model.add(Dense(1, kernel_initializer=kernel_init, activation='sigmoid'))
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model

In [14]:
if (notifyStatus): email_notify("Phase 2 Define Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 3. Compile Model

In [15]:
if (notifyStatus): email_notify("Phase 3 Compile Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [16]:
# Initialize the Keras model
cv_model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batches, verbose=0)

In [17]:
if (notifyStatus): email_notify("Phase 3 Compile Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 4. Fit Model

In [18]:
if (notifyStatus): email_notify("Phase 4 Fit Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [19]:
# Fit and evaluate the Keras model using 10-fold cross validation
kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seedNum)
results = cross_val_score(cv_model, X_train, y_train, cv=kfold)
print('Generating results using the metrics of', metrics)
print('All cross-Validate results:', results)
print('Average cross-Validate results:', results.mean())






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Generating results using the metrics of ['accuracy']
All cross-Validate results: [0.67796611 0.68965517 0.56896552 0.68965518 0.72413795 0.71929825
 0.75438596 0.73684211 0.63157895 0.71929825]
Average cross-Validate results: 0.6911783432942358


In [20]:
if (notifyStatus): email_notify("Phase 4 Fit Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 5. Evaluate Model

In [21]:
if (notifyStatus): email_notify("Phase 5 Evaluate Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [22]:
# Create the final model for evaluating the test dataset
model = create_model()
model.fit(X_train, y_train, epochs=epochs, batch_size=batches, verbose=1)

Epoch 1/150
576/576 [==============================] - 1s 2ms/step - loss: 4.0667 - acc: 0.4358
Epoch 2/150
576/576 [==============================] - 0s 348us/step - loss: 1.4050 - acc: 0.5573
Epoch 3/150
576/576 [==============================] - 0s 324us/step - loss: 0.9867 - acc: 0.5799
Epoch 4/150
576/576 [==============================] - 0s 319us/step - loss: 0.8767 - acc: 0.6111
Epoch 5/150
576/576 [==============================] - 0s 317us/step - loss: 0.8178 - acc: 0.6198
Epoch 6/150
576/576 [==============================] - 0s 319us/step - loss: 0.7817 - acc: 0.6111
Epoch 7/150
576/576 [==============================] - 0s 311us/step - loss: 0.7352 - acc: 0.6146
Epoch 8/150
576/576 [==============================] - 0s 322us/step - loss: 0.6869 - acc: 0.6302
Epoch 9/150
576/576 [==============================] - 0s 313us/step - loss: 0.6777 - acc: 0.6319
Epoch 10/150
576/576 [==============================] - 0s 324us/step - loss: 0.6445 - acc: 0.6563
Epoch 11/150
576/576 

In [23]:
# Evaluate the Keras model on previously unseen data
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))

192/192 [==============================] - 0s 1ms/step

acc: 78.12%

loss: 47.15%


In [24]:
if (notifyStatus): email_notify("Phase 5 Evaluate Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 6. Finalize Model

In [25]:
if (notifyStatus): email_notify("Phase 6 Finalize Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [26]:
# Make class predictions with the model
predictions = model.predict_classes(X_original)

# Summarize the first 20 cases
for i in range(20):
	print('%s => %d (expected %d)' % (X_original[i].tolist(), predictions[i], y_original[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 0 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 1 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 1 (expected 1)
[5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0] => 0 (expected 0)
[3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.248, 26.0] => 0 (expected 1)
[10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0] => 0 (expected 0)
[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0] => 1 (expected 1)
[8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.232, 54.0] => 0 (expected 1)
[4.0, 110.0, 92.0, 0.0, 0.0, 37.6, 0.191, 30.0] => 0 (expected 0)
[10.0, 168.0, 74.0, 0.0, 0.0, 38.0, 0.537, 34.0] => 1 (expected 1)
[10.0, 139.0, 80.0, 0.0, 0.0, 27.1, 1.441, 57.0] => 0 (expected 0)
[1.0, 189.0, 60.0, 23.0, 846.0, 30.1, 0.398, 59.0] => 1 (expected 1)
[5.0, 166.0, 72.0, 19.0, 175.0, 25.8, 0.587, 51.0] => 1 (expect

In [27]:
if (notifyStatus): email_notify("Phase 6 Finalize Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [28]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:04:07.515225
